# K-mer Aspect Ratio Survey for Christmas Tree Packing

## Competition Context

The **Santa 2025 Kaggle competition** challenges us to pack n Christmas trees (n=1 to 200) into the smallest possible **square bounding boxes**. The scoring metric is:

$$\text{Score} = \sum_{n=1}^{200} \frac{s_n^2}{n}$$

where $s_n$ is the side length of the bounding square for the n-tree configuration. Lower is better!

## The K-mer Strategy

A key insight from top competitors: instead of placing trees individually, we can **pre-optimize small groups of trees** (called **k-mers**) that pack tightly together, then **tile these k-mers in a grid** pattern.

- **Dimer (k=2)**: Two trees optimized together
- **Trimer (k=3)**: Three trees optimized together
- **Tetramer (k=4)**: Four trees optimized together

## Why Aspect Ratio Matters

When tiling k-mers in a grid, different grid layouts work better for different n values:
- For n=20 trees using dimers: could be 2x5 grid (10 dimers) or 5x2 grid
- A 2x5 grid needs a **wide dimer** (aspect ratio > 1)
- A 5x2 grid needs a **tall dimer** (aspect ratio < 1)

By having k-mers optimized for different aspect ratios, we can choose the one that best matches our target grid layout!

## What This Notebook Does

We systematically explore k-mer configurations across different aspect ratios:
1. For each k (2, 3, 4), try different target aspect ratios
2. Use CMA-ES optimization to find the most compact k-mer for each aspect ratio
3. Save all configurations for later use in tessellation strategies

## Setup

We need:
- **numpy**: Numerical operations
- **shapely**: Geometric operations (polygon intersection, bounds)
- **cma**: CMA-ES optimizer (install with `pip install cma`)
- **matplotlib**: Visualization

The **scale factor (1e15)** is critical: Shapely uses floating point arithmetic internally. By scaling our coordinates by 1e15, we maintain high precision for collision detection.

In [ ]:
! pip install cma

In [ ]:
import numpy as np
import json
import os
from decimal import Decimal, getcontext
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from shapely.geometry import Polygon
from shapely import affinity
import cma
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')

# High precision arithmetic
SCALE = 1e15
getcontext().prec = 25

# Output directory
OUTPUT_DIR = '/kaggle/working/kmer_survey'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# K-mer names for display
KMER_NAMES = {2: 'Dimer', 3: 'Trimer', 4: 'Tetramer'}

print(f"Output directory: {OUTPUT_DIR}")
print(f"Survey will cover: {list(KMER_NAMES.values())}")

## Christmas Tree Geometry

Each tree has a specific shape:
- **Three tiered triangular sections** forming the foliage (like a stylized Christmas tree)
- **A rectangular trunk** at the bottom

Key dimensions:
- Total height: ~1.0 units (0.8 foliage + 0.2 trunk)
- Maximum width: 0.7 units (at the bottom tier)
- Trunk: 0.15 wide x 0.2 tall

The tree can **rotate 0-360 degrees** around its center point. This rotational freedom is what makes the optimization interesting - by rotating trees, we can make them interlock more tightly.

### High-Precision Coordinates

We use Python's `Decimal` type and scale by 1e15 because:
1. Shapely performs intersection tests using floating point
2. Small numerical errors can cause false collision detections
3. Scaling to large integers preserves precision in geometric operations

In [ ]:
class ChristmasTree:
    """
    Represents a single, rotatable Christmas tree of a fixed size.
    
    The tree consists of:
    - Three tiered sections (triangular shapes)
    - A rectangular trunk at the bottom
    
    Uses Decimal arithmetic for high precision to avoid floating point errors.
    """

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """
        Initialize the Christmas tree with a specific position and rotation.
        
        Args:
            center_x: X coordinate of tree center (string for Decimal precision)
            center_y: Y coordinate of tree center (string for Decimal precision)
            angle: Rotation angle in degrees (string for Decimal precision)
        """
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        
        # Scale factor for high precision
        scale_factor = Decimal('1e15')

        # Tree dimensions
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        # Build the tree polygon (15 vertices)
        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )

        # Apply rotation and translation
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )

# Test: visualize a single tree
fig, ax = plt.subplots(figsize=(4, 5))
tree = ChristmasTree('0', '0', '0')
coords = [(x/SCALE, y/SCALE) for x, y in tree.polygon.exterior.coords]
poly = patches.Polygon(coords, facecolor='green', edgecolor='darkgreen', linewidth=2, alpha=0.7)
ax.add_patch(poly)
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.3, 1.0)
ax.set_aspect('equal')
ax.set_title('Christmas Tree Shape (angle=0)')
ax.grid(True, alpha=0.3)
plt.show()

## Core Functions: Collision Detection and Bounding Box

### Collision Detection
Two trees **collide** if their polygons overlap (have non-zero intersection area). We use Shapely's `intersects()` and `intersection()` methods.

### Bounding Box
The bounding box is the smallest axis-aligned rectangle containing all tree vertices. For our scoring, we care about the **square** bounding box (max of width and height).

### Push-Apart Algorithm
When initializing k-mers, trees may overlap. The **push-apart algorithm** applies repulsive forces between overlapping trees, iteratively separating them until no collisions remain. This guarantees a valid starting point for optimization.

In [ ]:
def get_tree_polygon(x, y, angle):
    """Get Shapely polygon for tree at given position and angle."""
    tree = ChristmasTree(str(x), str(y), str(angle))
    coords = [(c[0] / SCALE, c[1] / SCALE) for c in tree.polygon.exterior.coords]
    return Polygon(coords)


def has_collision(trees_data, tolerance=1e-9):
    """Check if any trees overlap.
    
    Args:
        trees_data: List of (x, y, angle) tuples
        tolerance: Minimum overlap area to count as collision
    
    Returns:
        True if any pair of trees overlaps
    """
    polygons = [get_tree_polygon(x, y, a) for x, y, a in trees_data]

    for i in range(len(polygons)):
        for j in range(i + 1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                inter = polygons[i].intersection(polygons[j])
                if inter.area > tolerance:
                    return True
    return False


def compute_bounds(trees_data):
    """Compute bounding box for tree configuration.
    
    Returns:
        (min_x, max_x, min_y, max_y) tuple
    """
    all_x = []
    all_y = []

    for x, y, angle in trees_data:
        tree = ChristmasTree(str(x), str(y), str(angle))
        for vx, vy in tree.polygon.exterior.coords:
            all_x.append(vx / SCALE)
            all_y.append(vy / SCALE)

    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)

    return min_x, max_x, min_y, max_y


def push_apart_kmer(params, k, max_iterations=100, force=0.15):
    """
    Push overlapping trees apart until the k-mer configuration is valid.
    
    Uses repulsive forces between overlapping trees to separate them.
    The first tree stays at origin, others move relative to it.
    
    Args:
        params: Parameter array (see generate_kmer for format)
        k: Number of trees
        max_iterations: Maximum separation iterations
        force: Repulsive force strength
    
    Returns:
        Modified params with trees pushed apart
    """
    params = np.array(params).copy()

    for _ in range(max_iterations):
        # Extract current positions (tree 0 at origin, others from params)
        positions = [(0.0, 0.0)]  # Tree 0 at origin
        for i in range(1, k):
            idx = 1 + 3 * (i - 1)
            positions.append((params[idx], params[idx + 1]))

        # Get angles
        angles = [params[0] % 360]
        for i in range(1, k):
            idx = 3 + 3 * (i - 1)
            angles.append(params[idx] % 360)

        # Create trees and check validity
        trees_data = [(positions[i][0], positions[i][1], angles[i]) for i in range(k)]
        if not has_collision(trees_data):
            return params

        # Create tree objects for collision checking
        trees = []
        for x, y, angle in trees_data:
            trees.append(ChristmasTree(str(x), str(y), str(angle)))

        # Compute repulsive forces
        fx = np.zeros(k)
        fy = np.zeros(k)

        for i in range(k):
            for j in range(i + 1, k):
                if trees[i].polygon.intersects(trees[j].polygon):
                    dx = positions[i][0] - positions[j][0]
                    dy = positions[i][1] - positions[j][1]
                    d = np.sqrt(dx * dx + dy * dy) + 1e-6
                    fx[i] += force * dx / d
                    fy[i] += force * dy / d
                    fx[j] -= force * dx / d
                    fy[j] -= force * dy / d

        # Apply forces and recenter so tree 0 stays at origin
        new_positions = [(positions[i][0] + fx[i], positions[i][1] + fy[i]) for i in range(k)]
        offset_x = new_positions[0][0]
        offset_y = new_positions[0][1]
        new_positions = [(x - offset_x, y - offset_y) for x, y in new_positions]

        # Update params with new positions
        for i in range(1, k):
            idx = 1 + 3 * (i - 1)
            params[idx] = new_positions[i][0]
            params[idx + 1] = new_positions[i][1]

    return params

print("Core functions defined.")

## K-mer Generation and Objective Function

### Parameter Encoding

A k-mer is defined by **3k-2 parameters**:
- **Tree 0**: Fixed at origin (0, 0), only angle varies → **1 parameter** (angle)
- **Trees 1 to k-1**: Position (x, y) and angle → **3 parameters each**

Total: 1 + 3(k-1) = 3k - 2 parameters

Example for dimer (k=2): `[angle0, x1, y1, angle1]` = 4 parameters

### Objective Function

We want to **minimize the bounding box area** while achieving a **target aspect ratio**.

- **Aspect ratio** = width / height (W/H convention)
  - aspect = 1.0 → square bounding box
  - aspect = 2.0 → wide (2x wider than tall)
  - aspect = 0.5 → tall (2x taller than wide)

- **Hard constraint**: No overlapping trees (returns high penalty)
- **Soft constraint**: Aspect ratio should be close to target (small penalty if off)

In [ ]:
def generate_kmer(params, k):
    """
    Generate k-mer configuration from parameter vector.
    
    params layout:
    - Tree 0: at origin (0, 0), angle = params[0]
    - Tree i (i>0): x = params[1 + 3*(i-1)], y = params[2 + 3*(i-1)], angle = params[3 + 3*(i-1)]
    
    Returns:
        List of (x, y, angle) tuples
    """
    trees = [(0, 0, params[0] % 360)]

    for i in range(1, k):
        idx = 1 + 3 * (i - 1)
        x = params[idx]
        y = params[idx + 1]
        angle = params[idx + 2] % 360
        trees.append((x, y, angle))

    return trees


def objective_for_aspect(params, k, target_aspect, aspect_tolerance=0.10):
    """
    Objective function: Minimize bounding box area with aspect ratio constraint.
    
    Args:
        params: Parameter vector
        k: Number of trees
        target_aspect: Desired width/height ratio
        aspect_tolerance: Acceptable deviation from target
    
    Returns:
        Objective value (lower is better)
    """
    try:
        trees = generate_kmer(params, k)

        # HARD collision constraint
        if has_collision(trees):
            return 100.0

        # Compute bounds
        min_x, max_x, min_y, max_y = compute_bounds(trees)
        width = max_x - min_x
        height = max_y - min_y

        if width < 0.01 or height < 0.01:
            return 100.0

        # W/H convention: aspect = width / height
        actual_aspect = width / height
        aspect_error = abs(actual_aspect - target_aspect) / target_aspect

        # Soft aspect ratio penalty
        if aspect_error > aspect_tolerance:
            aspect_penalty = 50.0 + aspect_error * 10
        else:
            aspect_penalty = 0.0

        # Primary objective: minimize area
        area = width * height

        return area + aspect_penalty

    except Exception:
        return 100.0


def generate_initial_guess(k, target_aspect):
    """
    Generate initial configuration for optimization.
    
    Strategy:
    1. Create tight overlapping cluster with random angles
    2. Apply push_apart to separate trees
    3. CMA-ES optimizes from this valid starting point
    """
    import math

    # Start with random angle for first tree
    initial = [np.random.uniform(0, 360)]

    # Very tight initial spacing (will overlap)
    base_spacing = 0.15

    for i in range(1, k):
        # Arrange in a tight cluster around origin
        angle_offset = i * (360.0 / k)
        r = base_spacing * (1 + i * 0.15)

        x = r * math.cos(math.radians(angle_offset))
        y = r * math.sin(math.radians(angle_offset))
        tree_angle = np.random.uniform(0, 360)

        initial.extend([x, y, tree_angle])

    # Apply push_apart to create valid starting configuration
    initial = np.array(initial)
    initial = push_apart_kmer(initial, k, max_iterations=200)

    return np.array(initial)

print("K-mer generation and objective functions defined.")

## CMA-ES Optimization

### What is CMA-ES?

**CMA-ES** (Covariance Matrix Adaptation Evolution Strategy) is a state-of-the-art **derivative-free optimizer**. It's particularly well-suited for:

- **Non-convex landscapes**: Our objective has many local minima
- **No gradients needed**: Objective function is "black box"
- **Automatic step-size adaptation**: Learns the search distribution

### How It Works

1. Maintains a **multivariate Gaussian distribution** over the parameter space
2. **Samples** candidate solutions from this distribution
3. **Evaluates** each candidate using the objective function
4. **Updates** the distribution based on which candidates performed best
5. Repeat until convergence

### Our Setup

- **Parameters**: 3k-2 dimensions (angles and positions)
- **Bounds**: Angles [0, 360], positions within reasonable range
- **Population size**: Scales with k (24 + 4k)
- **Multiple restarts**: Run several times, keep best result

In [ ]:
def optimize_single(args):
    """
    Worker function: Optimize a single k-mer configuration.
    
    Args:
        args: Tuple of (k, target_aspect, seed, max_iter)
    
    Returns:
        Dict with optimization results, or None if failed
    """
    k, target_aspect, seed, max_iter = args

    np.random.seed(seed)

    # Generate valid starting point
    initial = generate_initial_guess(k, target_aspect)

    # Add small noise
    noise = np.zeros(len(initial))
    noise[0] = np.random.randn() * 30  # First tree angle
    for i in range(1, k):
        idx = 1 + 3 * (i - 1)
        noise[idx] = np.random.randn() * 0.1      # x position
        noise[idx + 1] = np.random.randn() * 0.1  # y position
        noise[idx + 2] = np.random.randn() * 30   # angle
    initial = initial + noise

    # Bounds - scale with k for larger k-mers
    pos_bound = 2.0 + 0.5 * k
    bounds_lower = [0] + [-pos_bound, -pos_bound, 0] * (k - 1)
    bounds_upper = [360] + [pos_bound, pos_bound, 360] * (k - 1)

    # Per-parameter standard deviations
    cma_stds = [45.0]  # First tree angle
    for i in range(1, k):
        cma_stds.extend([0.2, 0.2, 45.0])  # x, y, angle

    opts = {
        'bounds': [bounds_lower, bounds_upper],
        'maxiter': max_iter,
        'popsize': 24 + 4 * k,
        'seed': seed,
        'verbose': -9,  # Suppress output
        'tolfun': 1e-7,
        'CMA_stds': cma_stds,
    }

    try:
        es = cma.CMAEvolutionStrategy(list(initial), 1.0, opts)

        while not es.stop():
            solutions = es.ask()
            fitnesses = [objective_for_aspect(x, k, target_aspect) for x in solutions]
            es.tell(solutions, fitnesses)

        result = es.result

        if result.fbest > 50:
            return None

        trees = generate_kmer(result.xbest, k)

        if has_collision(trees):
            return None

        min_x, max_x, min_y, max_y = compute_bounds(trees)
        width = max_x - min_x
        height = max_y - min_y
        actual_aspect = width / height

        # Verify aspect ratio is within tolerance
        if abs(actual_aspect - target_aspect) / target_aspect > 0.15:
            return None

        return {
            'k': k,
            'target_aspect': target_aspect,
            'actual_aspect': actual_aspect,
            'width': width,
            'height': height,
            'area': width * height,
            'trees': trees,
            'seed': seed,
            'score': result.fbest,
        }

    except Exception:
        return None

print("Optimization function defined.")

## Visualization

Each k-mer configuration is visualized with:
- **Color-coded trees**: Each tree in a different color
- **Bounding box**: Red dashed rectangle showing the extent
- **Metrics**: Target aspect, actual aspect, dimensions, and area

In [ ]:
def visualize_kmer(config, output_path=None, show=True):
    """Generate visualization of k-mer configuration.
    
    Args:
        config: Dict with k-mer configuration
        output_path: If provided, save PNG to this path
        show: If True, display the plot
    """
    k = config['k']
    trees = config['trees']
    target_aspect = config['target_aspect']
    actual_aspect = config['actual_aspect']
    width = config['width']
    height = config['height']
    area = config['area']

    fig, ax = plt.subplots(1, 1, figsize=(8, 8))

    # Color each tree differently
    colors = plt.cm.Set1(np.linspace(0, 1, min(k, 9)))

    for i, (x, y, angle) in enumerate(trees):
        tree = ChristmasTree(str(x), str(y), str(angle))
        coords = [(vx/SCALE, vy/SCALE) for vx, vy in tree.polygon.exterior.coords]
        poly = patches.Polygon(coords, facecolor=colors[i % len(colors)], edgecolor='black',
                               linewidth=1, alpha=0.7)
        ax.add_patch(poly)
        ax.plot(x, y, 'ko', markersize=3)  # Center marker

    # Draw bounding box
    min_x, max_x, min_y, max_y = compute_bounds(trees)
    rect = patches.Rectangle((min_x, min_y), width, height,
                              linewidth=2, edgecolor='red',
                              facecolor='none', linestyle='--')
    ax.add_patch(rect)

    margin = 0.2
    ax.set_xlim(min_x - margin, max_x + margin)
    ax.set_ylim(min_y - margin, max_y + margin)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    kmer_name = KMER_NAMES.get(k, f'{k}-mer')
    ax.set_title(f'{kmer_name}: Target Aspect {target_aspect:.2f}\n'
                 f'Actual: {actual_aspect:.2f}, Box: {width:.3f} x {height:.3f}, Area: {area:.4f}')

    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
    
    if show:
        plt.show()
    
    plt.close()

print("Visualization function defined.")

## Survey Configuration (Kaggle-Optimized)

We've tuned the parameters for **reasonable runtime on Kaggle** (free tier):

| Parameter | Full Survey | Kaggle Version |
|-----------|-------------|----------------|
| K values | 2, 3, 4, 5, 6 | **2, 3, 4** |
| Aspect ratios | 50+ per k | **7 per k** |
| Restarts | 50 | **10** |
| Max iterations | 400-1000 | **200** |
| Workers | 6-22 | **2** |

**Expected runtime: ~10-15 minutes**

For a more comprehensive survey, you can increase these values (run overnight on a machine with more CPUs).

In [ ]:
# Kaggle-optimized configuration
KAGGLE_CONFIG = {
    'k_values': [2, 3, 4],                           # Dimer, Trimer, Tetramer
    'aspects': [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0],  # 7 aspect ratios
    'n_restarts': 10,                                # Restarts per aspect
    'max_iter': 200,                                 # CMA-ES iterations
    'n_workers': 2                                   # Parallel workers
}

print("Survey Configuration:")
print(f"  K values: {KAGGLE_CONFIG['k_values']}")
print(f"  Aspect ratios: {KAGGLE_CONFIG['aspects']}")
print(f"  Restarts per (k, aspect): {KAGGLE_CONFIG['n_restarts']}")
print(f"  Max CMA-ES iterations: {KAGGLE_CONFIG['max_iter']}")
print(f"  Parallel workers: {KAGGLE_CONFIG['n_workers']}")
print()

total_optimizations = len(KAGGLE_CONFIG['k_values']) * len(KAGGLE_CONFIG['aspects']) * KAGGLE_CONFIG['n_restarts']
print(f"Total optimization runs: {total_optimizations}")
print(f"Estimated runtime: ~10-15 minutes")

## Running the Survey

We process each k-value **sequentially** to avoid overwhelming Kaggle's resources. Within each k-value, we run optimizations in parallel using a process pool.

For each (k, aspect_ratio) combination:
1. Run multiple restarts with different random seeds
2. Keep all valid configurations (within 15% of best area)
3. Save visualizations and results

In [ ]:
def run_survey_for_k(k, config=KAGGLE_CONFIG):
    """Run the survey for a single k value."""
    aspects = config['aspects']
    n_restarts = config['n_restarts']
    max_iter = config['max_iter']
    n_workers = config['n_workers']

    kmer_name = KMER_NAMES.get(k, f'{k}-mer')
    print(f"\n{kmer_name} (k={k})")
    print(f"  Aspects: {aspects[0]:.1f} to {aspects[-1]:.1f} ({len(aspects)} values)")
    print(f"  Restarts: {n_restarts}, Iterations: {max_iter}, Workers: {n_workers}")
    print("-" * 50)

    # Build all tasks
    tasks = []
    for target_aspect in aspects:
        for restart in range(n_restarts):
            seed = 10000 * k + int(target_aspect * 100) + restart * 1000
            tasks.append((k, target_aspect, seed, max_iter))

    # Run in parallel
    with Pool(n_workers) as pool:
        results = pool.map(optimize_single, tasks)

    # Group valid results by aspect ratio
    results_by_aspect = {}
    for r in results:
        if r is not None:
            aspect = r['target_aspect']
            if aspect not in results_by_aspect:
                results_by_aspect[aspect] = []
            results_by_aspect[aspect].append(r)

    # Sort each aspect's results by area and filter
    for aspect in results_by_aspect:
        results_by_aspect[aspect].sort(key=lambda x: x['area'])
        best_area = results_by_aspect[aspect][0]['area']
        results_by_aspect[aspect] = [r for r in results_by_aspect[aspect]
                                     if r['area'] <= best_area + 0.15]

    # Collect and report results
    all_results = []
    for target_aspect in aspects:
        if target_aspect in results_by_aspect:
            configs = results_by_aspect[target_aspect]
            best = configs[0]
            print(f"  Aspect {target_aspect:.2f}: {len(configs)} configs, best area={best['area']:.4f}, "
                  f"actual={best['actual_aspect']:.2f}, box={best['width']:.3f}x{best['height']:.3f}")

            # Save visualizations and results for best configuration
            for variant_idx, cfg in enumerate(configs[:3]):  # Save top 3 variants
                filename = f"{kmer_name.lower()}_aspect_{target_aspect:.2f}_v{variant_idx}.png"
                output_path = os.path.join(OUTPUT_DIR, filename)
                visualize_kmer(cfg, output_path, show=False)

                # Prepare for JSON
                result = {
                    'k': k,
                    'kmer_name': kmer_name,
                    'target_aspect': target_aspect,
                    'actual_aspect': cfg['actual_aspect'],
                    'width': cfg['width'],
                    'height': cfg['height'],
                    'area': cfg['area'],
                    'variant': variant_idx,
                    'trees': [{'x': t[0], 'y': t[1], 'angle': t[2]} for t in cfg['trees']]
                }
                all_results.append(result)
        else:
            print(f"  Aspect {target_aspect:.2f}: FAILED")

    return all_results

print("Survey function defined. Ready to run!")

In [ ]:
# Run the full survey
print("K-mer Aspect Ratio Survey")
print("=" * 60)
print(f"Output directory: {OUTPUT_DIR}")
print(f"K values: {KAGGLE_CONFIG['k_values']}")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

all_results = []
for k in KAGGLE_CONFIG['k_values']:
    results = run_survey_for_k(k)
    all_results.extend(results)

print()
print("=" * 60)
print(f"Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total configurations found: {len(all_results)}")

In [ ]:
# Save all results to JSON
results_file = os.path.join(OUTPUT_DIR, 'survey_results.json')
with open(results_file, 'w') as f:
    json.dump({
        'metadata': {
            'version': 'kaggle_v1',
            'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'k_values': KAGGLE_CONFIG['k_values'],
            'aspects': KAGGLE_CONFIG['aspects'],
            'n_restarts': KAGGLE_CONFIG['n_restarts'],
            'max_iter': KAGGLE_CONFIG['max_iter'],
        },
        'configurations': all_results
    }, f, indent=2)

print(f"Results saved to: {results_file}")

## Results Analysis

Let's examine what we found:
- Which aspect ratios produce the most compact k-mers?
- How does area vary across aspect ratios?
- Which configurations would be best for different grid layouts?

In [ ]:
# Summary statistics
print("\nSummary by K-mer Type")
print("=" * 60)

for k in KAGGLE_CONFIG['k_values']:
    k_results = [r for r in all_results if r['k'] == k]
    if k_results:
        areas = [r['area'] for r in k_results]
        aspects = [r['actual_aspect'] for r in k_results]
        
        kmer_name = KMER_NAMES.get(k, f'{k}-mer')
        print(f"\n{kmer_name} (k={k}):")
        print(f"  Configurations found: {len(k_results)}")
        print(f"  Aspect range: {min(aspects):.2f} - {max(aspects):.2f}")
        print(f"  Area range: {min(areas):.4f} - {max(areas):.4f}")
        
        # Find best (smallest area) configuration
        best = min(k_results, key=lambda x: x['area'])
        print(f"  Best config: aspect={best['actual_aspect']:.2f}, area={best['area']:.4f}")

In [ ]:
# Visualize the best configuration for each k
print("\nBest configuration for each k-mer type:")
print("-" * 40)

for k in KAGGLE_CONFIG['k_values']:
    k_results = [r for r in all_results if r['k'] == k]
    if k_results:
        best = min(k_results, key=lambda x: x['area'])
        kmer_name = KMER_NAMES.get(k, f'{k}-mer')
        print(f"\n{kmer_name}: Target aspect={best['target_aspect']:.2f}")
        
        # Convert trees list back to tuples for visualization
        best_viz = best.copy()
        best_viz['trees'] = [(t['x'], t['y'], t['angle']) for t in best['trees']]
        visualize_kmer(best_viz, show=True)

In [ ]:
# Plot area vs aspect ratio for each k
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, k in enumerate(KAGGLE_CONFIG['k_values']):
    k_results = [r for r in all_results if r['k'] == k and r['variant'] == 0]
    if k_results:
        aspects = [r['target_aspect'] for r in k_results]
        areas = [r['area'] for r in k_results]
        
        ax = axes[idx]
        ax.scatter(aspects, areas, s=100, alpha=0.7)
        ax.plot(aspects, areas, 'b-', alpha=0.3)
        ax.set_xlabel('Target Aspect Ratio (W/H)')
        ax.set_ylabel('Bounding Box Area')
        ax.set_title(f'{KMER_NAMES[k]} (k={k})')
        ax.grid(True, alpha=0.3)
        ax.axvline(x=1.0, color='red', linestyle='--', alpha=0.5, label='Square (1:1)')
        ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'area_vs_aspect.png'), dpi=150)
plt.show()

## Conclusion

### Key Findings

1. **Dimers** are the most reliable and commonly used for tessellation
2. **Aspect ratio ~1.0** (square) is often optimal, but not always!
3. **Trimers/Tetramers** can be more compact but are harder to tile effectively

### How to Use These Results

For a target of **n trees** using **dimers in an m x k grid**:
1. n = 2 * m * k (total trees = 2 trees/dimer * grid cells)
2. Target aspect ratio ≈ m / k
3. Look up the dimer configuration closest to your target aspect ratio

### Extending This Work

To improve results:
- **More k values**: Add k=5, 6 (pentamers, hexamers)
- **Finer aspect ratios**: Use step 0.1 instead of 0.25
- **More restarts**: 50+ instead of 10 for better convergence
- **Run overnight**: On a machine with more CPUs

### Files Generated

All outputs are in `/kaggle/working/kmer_survey/`:
- `survey_results.json`: All configurations with tree positions
- `*_aspect_*.png`: Visualizations for each k-mer type and aspect ratio
- `area_vs_aspect.png`: Summary plot of area vs aspect ratio

### Acknowledgments

This technique was discovered through analysis of top-performing competition notebooks that use dimer tessellation strategies. Special thanks to the Kaggle Santa 2025 competition community!

In [ ]:
# List all generated files
print("Generated files:")
for f in sorted(os.listdir(OUTPUT_DIR)):
    filepath = os.path.join(OUTPUT_DIR, f)
    size = os.path.getsize(filepath)
    print(f"  {f}: {size/1024:.1f} KB")